# Segmentation metrics evaluation

<ul>
    <li>Per Pixel Accuracy</li>
    <li>Per Class Accuracy</li>
    <li>Intersection over Union</li>
</ul>


In [2]:
import os
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from FCN.citydataset import classes_city

In [4]:
maps_classes = np.array([
    [255,255,251],
    [203,222,174],
    [171,208,251],
    [231,229,224],
    [243,239,235],
    [255,150,63]
])

facades_classes = np.array([
    [255,154,47],
    [194,0,47],
    [0,56,248],
    [252,766,30],
    [0,247,238],
    [0,129,249],
    [101,255,160],
    [197,2533,90],
    [0,24,215]
])

In [5]:
N = lambda i,j,truth, pred: np.sum(np.multiply(truth==i, pred==j))
T = lambda i, truth: np.sum(truth==i)

def PerPixelAccuracy(truth, pred, classes=maps_classes):
    num = np.sum([N(i,i,truth,pred) for i in range(len(classes))])
    den = np.sum([T(i,truth) for i in range(len(classes))])
    return num*1.0/den

def MeanAccuracy(truth, pred, classes=maps_classes):
    return np.sum([N(i,i,truth,pred)*1.0/(T(i,truth)+1e-15) for i in range(len(classes))])*1.0/(len(classes))


def MeanIU(truth, pred, classes=maps_classes):
    coef = np.sum([N(i,i,truth,pred)\
                   /(1e-15+T(i,truth)-N(i,i,truth, pred)\
                     +np.sum([N(j,i,truth,pred) for j in range(len(classes))]))\
                   for i in range(len(classes))])
    return coef*1.0/(len(classes))

def classif_score(idx,folder_model,fake_b,real_b, classes = maps_classes):
    F = lambda im: os.path.join(folder_model,fake_b[im])
    R = lambda im: os.path.join(folder_model,fake_b[im].replace('fake','real'))
    f = imread(F(idx)).reshape(-1,3)
    r = imread(R(idx)).reshape(-1,3)
    def find_cluster(vec, classes=classes):
        rscores = np.zeros((256*256,len(classes)))
        for i in range(len(classes)):
            rscores[:,i] = np.linalg.norm(vec-np.repeat(classes[i].reshape(1,3),256*256,axis = 0), axis = 1)
        vc = np.argmin(rscores, axis = 1)
        return vc
    pred = find_cluster(f)
    truth = find_cluster(r)
    ppa = PerPixelAccuracy(truth=truth,pred=pred, classes=classes),
    ma = MeanAccuracy(truth=truth,pred=pred, classes=classes),
    miu = MeanIU(truth=truth,pred=pred, classes=classes),
    return np.array([ppa, ma, miu]).reshape(1,3)

In [6]:
def Eval(model, classes):
    folder_model = r"results/{}/test_latest/images/".format(model)
    
    fake_b = sorted(list(filter(lambda x: 'fake_B' in x, os.listdir(folder_model))))
    real_b = sorted(list(filter(lambda x: 'real_B' in x, os.listdir(folder_model))))
    
    results = np.vstack((classif_score(idx,folder_model,fake_b,real_b,classes=classes) for idx in range(len(fake_b))))
    metrics = ['PerPixelAccuracy', 'MeanAccuracy','MeanIU']
    return dict(zip(metrics,results.mean(axis = 0))), dict(zip(metrics,results.std(axis = 0)))

## Maps <-> Aerial Photo

In [7]:
Eval("maps_cyclegan", classes = maps_classes)

({'PerPixelAccuracy': 0.65925537109375,
  'MeanAccuracy': 0.3657738185997323,
  'MeanIU': 0.26727131283783256},
 {'PerPixelAccuracy': 0.13116208723265588,
  'MeanAccuracy': 0.05745705406288182,
  'MeanIU': 0.059864040411428454})

In [8]:
Eval("maps_cyclegan_sn", classes = maps_classes)

({'PerPixelAccuracy': 0.6366470336914063,
  'MeanAccuracy': 0.34263235736806297,
  'MeanIU': 0.24193643097026268},
 {'PerPixelAccuracy': 0.13789364411105787,
  'MeanAccuracy': 0.04650520926674163,
  'MeanIU': 0.05943920037964254})

In [9]:
Eval("maps_cyclegan_wgangp", classes = maps_classes)

({'PerPixelAccuracy': 0.47609405517578124,
  'MeanAccuracy': 0.20456466436887275,
  'MeanIU': 0.12403574672476622},
 {'PerPixelAccuracy': 0.08503779849375542,
  'MeanAccuracy': 0.021947402156263413,
  'MeanIU': 0.02170079132551184})

## Façades

In [10]:
Eval("facades_cyclegan", classes = facades_classes)

({'PerPixelAccuracy': 0.2937838745117187,
  'MeanAccuracy': 0.15467122617863158,
  'MeanIU': 0.08586958203181137},
 {'PerPixelAccuracy': 0.12346242577135777,
  'MeanAccuracy': 0.04772577980802215,
  'MeanIU': 0.033463657767945666})

In [11]:
Eval("facades_cyclegan_sn", classes = facades_classes)

({'PerPixelAccuracy': 0.21249053955078126,
  'MeanAccuracy': 0.10819630133606947,
  'MeanIU': 0.05724362417167473},
 {'PerPixelAccuracy': 0.09118839294120668,
  'MeanAccuracy': 0.03377176683613442,
  'MeanIU': 0.020359784478120084})

In [12]:
Eval("facades_cyclegan_wgan_gp", classes = facades_classes)

({'PerPixelAccuracy': 0.23826690673828124,
  'MeanAccuracy': 0.13977146016789235,
  'MeanIU': 0.06841433039960275},
 {'PerPixelAccuracy': 0.07565204451374208,
  'MeanAccuracy': 0.032118218510339386,
  'MeanIU': 0.020692653008087075})

## CityScapes

In [13]:
file = r"datasets/cityscapes/testB/100_B.jpg"

In [14]:
Eval("cityscapes_cyclegan", classes = classes_city)

({'PerPixelAccuracy': 0.5474249267578125,
  'MeanAccuracy': 0.1890281816354468,
  'MeanIU': 0.13021065980625898},
 {'PerPixelAccuracy': 0.0850494451540961,
  'MeanAccuracy': 0.031338250596741134,
  'MeanIU': 0.024629139900385264})

In [15]:
Eval("cityscapes_cyclegan_sn", classes = classes_city)

({'PerPixelAccuracy': 0.533218994140625,
  'MeanAccuracy': 0.1823593079982619,
  'MeanIU': 0.12117730109861173},
 {'PerPixelAccuracy': 0.08950081338900766,
  'MeanAccuracy': 0.03357061966228876,
  'MeanIU': 0.025711568382825872})

In [16]:
Eval("cityscapes_cyclegan_wgan_gp", classes = classes_city)

({'PerPixelAccuracy': 0.4925592041015625,
  'MeanAccuracy': 0.13448065287914016,
  'MeanIU': 0.08848661403506419},
 {'PerPixelAccuracy': 0.07864055538098615,
  'MeanAccuracy': 0.01830047264505408,
  'MeanIU': 0.012299909018102231})